# Capstone Project - Segmenting and Clustering Neighborhoods in Bogota, Colombia

## Library instalation

In [1]:
%%capture
!pip install pandas      
!pip install numpy
!pip install requests
!pip install html5lib
!pip install bs4
!pip install geopy
!pip install folium
!pip install matplotlib

Create a data frame

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#from bs4 import BeautifulSoup
import os

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

import seaborn as sns

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
CLIENT_ID = 'BZHR4VZFYMXKFMU2OTRUU52LVEYH1SSQMTJ1FXOGPA5LPD3E'
CLIENT_SECRET = 'OH3QP0YZAPQJX1QIHXBZ15FQO2GWATKU1HNUW3RRUFPZZ25W'
VERSION = '20180605'

In [4]:
Bogota_df = pd.read_csv("CO1.csv", delimiter=",")
Bogota_df.head()

,Postal Code,Neighborhood,City,Latitude,Longitude
0,110111,Usaquen_Escuela de Caballería I,Bogota,4.6882,-74.0298
1,110121,Usaquen_Acacias Usaquen,Bogota,4.7136,-7.4033
2,110131,Usaquen_Barrancas,Bogota,4.7351,-74.0263
3,110141,Usaquen_Buenavista,Bogota,4.7648,-74.0287
4,110151,Usaquen_Torca I,Bogota,4.8044,-74.0222


In [5]:
Bogota_df.dtypes

Postal Code       int64
Neighborhood     object
City             object
Latitude        float64
Longitude       float64
dtype: object

In [6]:
from geopy.geocoders import Nominatim
address = 'Bogota, Colombia'
geolocator = Nominatim(user_agent="Bogota")
location = geolocator.geocode(address)
latitude_Bogota = location.latitude
longitude_Bogota = location.longitude
print('The geograpical coordinate of Bogota are {}, {}.'.format(latitude_Bogota, longitude_Bogota))

The geograpical coordinate of Bogota are 4.6533326, -74.083652.


In [7]:
import folium
map_bogota = folium.Map(location=[latitude_Bogota,longitude_Bogota],zoom_start=10)
for LATITUD,lng,borough,neighborhood in zip(Bogota_df['Latitude'],Bogota_df['Longitude'],Bogota_df['Neighborhood'],Bogota_df['Postal Code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [LATITUD,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_bogota)
map_bogota

### Run k-means to cluster the neighborhood into 5 clusters

In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
Bogota_venues = getNearbyVenues(names=Bogota_df['Neighborhood'],latitudes=Bogota_df['Latitude'],longitudes=Bogota_df['Longitude'])

Usaquen_Escuela de Caballería I
Usaquen_Acacias Usaquen
Usaquen_Barrancas
Usaquen_Buenavista
Usaquen_Torca I
Chapinero_Esperanza
Chapinero_Antiguo Country
Chapinero_Bosque Calderón
Santa Fe_Bosque Izquierdo
Santa Fe_El Dorado
San Cristobal_Aguas Claras
San Cristobal_Atenas Sur
San Cristobal_Altamira
San Cristobal_Canadá o Guira
Usme_Granada Sur
Usme_Alaska
Usme_Brisas del Llano
Usme_Antonio José de Sucre
Usme_Olarte
Usme_
Usme_Veredas Chisaca
Tunjuelito_El Carmen
Tunjuelito_Abraham Lincoln
Bosa_Betania
Bosa_Bosa Nova
Bosa_Antonia Santos
Bosa_Andalucía II
Kennedy_El Tintal III
Kennedy_Bavaria
Kennedy_Cooperativa de Sub-Oficiales
Kennedy_Alqueria La Fragua
Kennedy_Chucua de La Vaca I
Kennedy_Casa Blanca Sur
Kennedy_Calandaima
Kennedy_Ciudad de Cali
Fontibon_Aeropuerto El Dorado
Fontibon_Belén Fontibón
Engativa_Bochica II
Engativa_Autopista Medellín
Engativa_Bolivia
Engativa_Alamos
Engativa_Boyacá
Engativa_Bellavista Occidental
Engativa_El Encanto
Suba_Batán
Suba_Aures
Suba_Aures II
Suba_

In [10]:
Bogota_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquen_Escuela de Caballería I,4.6882,-74.0298,Parque Publico Usaquén II,4.691297,-74.031994,Dog Run
1,Usaquen_Escuela de Caballería I,4.6882,-74.0298,Hotel NH Collection Bogotá Hacienda Royal,4.691981,-74.031946,Hotel
2,Usaquen_Escuela de Caballería I,4.6882,-74.0298,Sabor latino,4.690271,-74.029366,Food Truck
3,Usaquen_Escuela de Caballería I,4.6882,-74.0298,Coshelo,4.684603,-74.030592,Fast Food Restaurant
4,Usaquen_Escuela de Caballería I,4.6882,-74.0298,Cancha de Tennis Santa Ana,4.685126,-74.032913,Tennis Court


In [11]:
print('{} venues were returned by Foursquare.'.format(Bogota_venues.shape[0]))

334 venues were returned by Foursquare.


In [12]:
Bogota_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Antonio Nariño_Caracas,1,1,1,1,1,1
Bosa_Andalucía II,3,3,3,3,3,3
Bosa_Antonia Santos,2,2,2,2,2,2
Bosa_Betania,2,2,2,2,2,2
Candelaria_Belén,100,100,100,100,100,100
Chapinero_Antiguo Country,3,3,3,3,3,3
Chapinero_Esperanza,2,2,2,2,2,2
Ciudad Bolivar_Arborizadora Baja,4,4,4,4,4,4
Ciudad Bolivar_Barlovento,3,3,3,3,3,3


In [13]:
print('There are {} uniques categories.'.format(len(Bogota_venues['Venue Category'].unique())))

There are 113 uniques categories.


In [14]:
# one hot encoding
Bogota_onehot = pd.get_dummies(Bogota_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bogota_onehot['Neighborhood'] = Bogota_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bogota_onehot.columns[-1]] + list(Bogota_onehot.columns[:-1])
Bogota_onehot.head()

,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Building,Burger Joint,Bus Station,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Event Service,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Greek Restaurant,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Market,Martial Arts School,Mexican Restaurant,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Other Repair Shop,Park,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Recreation Center,Restaurant,Rock Club,Salad Place,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Snack Place,Soccer Field,South American Restaurant,State / Provincial Park,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Theater,Vegetarian / Vegan Restaurant,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Usaquen_Escuela de Caballería I,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Usaquen_Escuela de Caballería I,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Usaquen_Escuela de Caballería I,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Usaquen_Escuela de Caballería I,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Usaquen_Escuela de Caballería I,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [15]:
Bogota_grouped = Bogota_onehot.groupby('Neighborhood').mean().reset_index()
Bogota_grouped

,Neighborhood,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Building,Burger Joint,Bus Station,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Event Service,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Greek Restaurant,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Market,Martial Arts School,Mexican Restaurant,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Museum,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Optical Shop,Other Repair Shop,Park,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Recreation Center,Restaurant,Rock Club,Salad Place,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Snack Place,Soccer Field,South American Restaurant,State / Provincial Park,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Theater,Vegetarian / Vegan Restaurant,Wings Joint
0,Antonio Nariño_Caracas,0.00,0.00,0.00000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.00000,0.00,0.00000,0.00,0.00000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.00000,0.000,0.000000,0.00,0.00
1,Bosa_Andalucía II,0.00,0.00,0.00000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.333333,0.00,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.333333,0.000000,0.0,0.0,0.00,0.00,0.00000,0.00,0.00000,0.00,0.00000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.00000,0.000,0.000000,0.00,0.00
2,Bosa_Antonia Santos,0.00,0.00,0.00000,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.500000,0.00,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.00000,0.00,0.00000,0.00,0.00000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000

In [16]:
num_top_venues = 5

for hood in Bogota_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Bogota_grouped[Bogota_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Antonio Nariño_Caracas----
               venue  freq
0              Diner   1.0
1             Arcade   0.0
2             Hostel   0.0
3               Park   0.0
4  Other Repair Shop   0.0


----Bosa_Andalucía II----
                 venue  freq
0   Mexican Restaurant  0.33
1           Restaurant  0.33
2  Fried Chicken Joint  0.33
3    Mobile Phone Shop  0.00
4    Other Repair Shop  0.00


----Bosa_Antonia Santos----
                 venue  freq
0           Restaurant   0.5
1  Fried Chicken Joint   0.5
2               Arcade   0.0
3    Mobile Phone Shop   0.0
4    Other Repair Shop   0.0


----Bosa_Betania----
                venue  freq
0       Movie Theater   0.5
1       Shopping Mall   0.5
2              Arcade   0.0
3  Mexican Restaurant   0.0
4   Other Repair Shop   0.0


----Candelaria_Belén----
                venue  freq
0                Café  0.08
1          Restaurant  0.07
2              Hostel  0.07
3         Coffee Shop  0.05
4  Italian Restaurant  0.05


----Chapinero

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bogota_grouped['Neighborhood']

for ind in np.arange(Bogota_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bogota_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Antonio Nariño_Caracas,Diner,Wings Joint,Vegetarian / Vegan Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Dog Run,Donut Shop
1,Bosa_Andalucía II,Fried Chicken Joint,Restaurant,Mexican Restaurant,Wings Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop
2,Bosa_Antonia Santos,Fried Chicken Joint,Restaurant,Wings Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
3,Bosa_Betania,Movie Theater,Shopping Mall,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
4,Candelaria_Belén,Café,Hostel,Restaurant,Coffee Shop,Italian Restaurant,Breakfast Spot,Plaza,Latin American Restaurant,BBQ Joint,Gastropub


In [19]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Bogota_grouped_clustering = Bogota_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bogota_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 3, 4, 4, 4], dtype=int32)

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

In [21]:
Bogota_merged = Bogota_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Bogota_merged = Bogota_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Bogota_merged.head() # check the last columns!

,Postal Code,Neighborhood,City,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,110111,Usaquen_Escuela de Caballería I,Bogota,4.6882,-74.0298,4.0,Tennis Court,Hotel,Dog Run,Food Truck,Fast Food Restaurant,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center
1,110121,Usaquen_Acacias Usaquen,Bogota,4.7136,-7.4033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110131,Usaquen_Barrancas,Bogota,4.7351,-74.0263,4.0,Salad Place,Skating Rink,Fast Food Restaurant,Restaurant,Flea Market,Coffee Shop,Seafood Restaurant,Food & Drink Shop,Dessert Shop,Fried Chicken Joint
3,110141,Usaquen_Buenavista,Bogota,4.7648,-74.0287,4.0,Market,Bus Station,Bakery,Fast Food Restaurant,Food,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
4,110151,Usaquen_Torca I,Bogota,4.8044,-74.0222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# There are empty cells that impide to continue the analysis, then it's necessary to drop out empty cells
Bogota_merged = Bogota_merged.dropna()

In [23]:
# It's necessary to get the cluster labels in "int" format
Bogota_merged['Cluster_Labels'] = Bogota_merged.Cluster_Labels.astype(int)

In [24]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude_Bogota, longitude_Bogota], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bogota_merged['Latitude'], Bogota_merged['Longitude'], Bogota_merged['Neighborhood'], Bogota_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [25]:
Bogota_merged.loc[Bogota_merged['Cluster_Labels'] == 0, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Santa Fe_Bosque Izquierdo,0,Ice Cream Shop,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run


Cluster 2

In [26]:
Bogota_merged.loc[Bogota_merged['Cluster_Labels'] == 1, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,San Cristobal_Atenas Sur,1,Gym / Fitness Center,Playground,Wings Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
21,Tunjuelito_El Carmen,1,Gym / Fitness Center,Café,Wings Joint,Flea Market,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run
38,Engativa_Autopista Medellín,1,Gym / Fitness Center,Department Store,Dessert Shop,BBQ Joint,Bakery,Wings Joint,Food,Convenience Store,Cultural Center,Diner
59,Puente Aranda_Barcelona,1,Pool,BBQ Joint,Bar,Wings Joint,Flea Market,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
66,Rafael Uribe Uribe_Arrayanes III,1,Supermarket,BBQ Joint,Event Service,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop


Cluster 3

In [27]:
Bogota_merged.loc[Bogota_merged['Cluster_Labels'] == 2, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Tunjuelito_Abraham Lincoln,2,Park,Café,Wings Joint,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run
45,Suba_Aures,2,Auto Garage,Park,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
47,Suba_Berlín,2,Other Repair Shop,Park,Concert Hall,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run
64,Rafael Uribe Uribe_Granja de Santa Sofía,2,Arepa Restaurant,Park,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner


Cluster 4

In [28]:
Bogota_merged.loc[Bogota_merged['Cluster_Labels'] == 3, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Chapinero_Esperanza,3,Mountain,Wings Joint,Health & Beauty Service,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run


Cluster 5

In [29]:
Bogota_merged.loc[Bogota_merged['Cluster_Labels'] == 4, Bogota_merged.columns[[1] + list(range(5, Bogota_merged.shape[1]))]]

,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Usaquen_Escuela de Caballería I,4,Tennis Court,Hotel,Dog Run,Food Truck,Fast Food Restaurant,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center
2,Usaquen_Barrancas,4,Salad Place,Skating Rink,Fast Food Restaurant,Restaurant,Flea Market,Coffee Shop,Seafood Restaurant,Food & Drink Shop,Dessert Shop,Fried Chicken Joint
3,Usaquen_Buenavista,4,Market,Bus Station,Bakery,Fast Food Restaurant,Food,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
6,Chapinero_Antiguo Country,4,Gym,Hotel,Athletics & Sports,Wings Joint,Flea Market,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
13,San Cristobal_Canadá o Guira,4,State / Provincial Park,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner,Dog Run
15,Usme_Alaska,4,Bus Station,Restaurant,Wings Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
23,Bosa_Betania,4,Movie Theater,Shopping Mall,Wings Joint,Flea Market,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
25,Bosa_Antonia Santos,4,Fried Chicken Joint,Restaurant,Wings Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop,Diner
26,Bosa_Andalucía II,4,Fried Chicken Joint,Restaurant,Mexican Restaurant,Wings Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Cultural Center,Department Store,Dessert Shop
28,Kennedy_Bavaria,4,Seafood Restaurant,BBQ Joint,Pizza Place,Asian Restaurant,Recreation Center,Pub,Fast Food Restaurant,Dessert Shop,Department Store,Falafel Restaurant
